In [6]:
import pandas as pd
import os
from sqlalchemy import create_engine
import logging
import time

In [7]:
logging.basicConfig(
    filename="logs/ingestion_db.log",
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filemode='a'
)


In [2]:
engine = create_engine('sqlite:///inventory.db')

In [8]:
def ingest_db(df, table_name, engine, chunk_size=1000):
    '''this function will ingest the dataframe into database table'''
    try:
        df.to_sql(
            table_name,
            con=engine,
            if_exists='replace',
            index=False,
            chunksize=chunk_size,
            method='multi'  # optional: speeds up insert for supported DBs
        )
        print(f"✅ Data ingested into {table_name} table.")
    except Exception as e:
        print(f"❌ Failed to ingest {table_name}: {e}")

In [9]:
def load_raw_data():
    '''this function will load the CSVs as dataframe and ingest into db'''
    start = time.time()
    for file in os.listdir('data'):
        if '.csv' in file:
            df = pd.read_csv(os.path.join('data', file))
            logging.info(f'Ingesting {file} in db')
            ingest_db(df, file.split('.')[0], engine)
        end = time.time()
        total_time = (end - start)/60
        logging.info('--------------Ingestion Complete---------------')
        logging.info(f'Total Time Taken: {total_time} minutes')

In [10]:
if __name__ == '__main__':
    load_raw_data()

✅ Data ingested into begin_inventory table.
✅ Data ingested into end_inventory table.
✅ Data ingested into purchases table.
✅ Data ingested into purchase_prices table.
✅ Data ingested into sales table.
✅ Data ingested into vendor_invoice table.
